In [67]:
import vrep 
import sys
import time 
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

from tank import *

In [68]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [69]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [70]:

def first_phase_logic():
    # for now let's ignore spped 
    
   
    # rules
    in_wn = ctrl.Antecedent(np.arange(1.5,2.8,0.05),'in_wn')
    out_velo = ctrl.Consequent(np.arange(0,7.1,0.1),'out_velo')
    out_velo.defuzzify_method = 'centroid'
    in_wn['low'] = fuzz.trimf(in_wn.universe,[1.6,1.6,1.8])
    in_wn['medium'] = fuzz.trimf(in_wn.universe,[1.6,1.8,2.8])
    in_wn['high'] = fuzz.trimf(in_wn.universe,[1.8,2.8,2.8])
    out_velo['low'] = fuzz.trimf(out_velo.universe,[0,0,2.5])

    out_velo['medium'] = fuzz.trimf(out_velo.universe,[0,2.5,4.5])

    out_velo['high'] = fuzz.trimf(out_velo.universe,[4.5,7,7])
    
    acc_rule = ctrl.Rule(antecedent= in_wn['low'] ,consequent = out_velo['low'])

    rul0 = ctrl.Rule(antecedent=(in_wn['medium'] ),consequent=out_velo['medium'])

    dec_rule = ctrl.Rule(antecedent=in_wn['high'],consequent=out_velo['high'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [71]:
step1 = first_phase_logic()

In [72]:

def detect_spot(step1 = step1):
    counter = 0
    WN = 0
    WS = 0
    previous_WN = 10
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
            if(err_code == 0): 
                if sensor_name =='WN':
               
                    WN = np.linalg.norm(detectedPoint)
                   
                if sensor_name =='WS':
                    
                    WS = np.linalg.norm(detectedPoint)
                    
        counter +=1 
        if counter % 10000== 0:
            print("Proximity_sensor WN",WN,"Velocity",tank.rightvelocity)
            print("Proximity_sensor WS",WS,"Velocity",tank.rightvelocity)

                            
        try:
            if previous_WN > WN:
                step1.input['in_wn'] = WN
                step1.compute()
                tank.forward(step1.output['out_velo'])
            if ((WN - WS >0.15)) :
               tank.stop()
               return
        except:
            continue
    

In [73]:
# detect_spot()

In [74]:

def rotate_phase_logic():
    # for now let's ignore spped 
    
   
    # rules
    in_nw = ctrl.Antecedent(np.arange(1.5,2.5,0.05),'in_nw')
    out_velo_delta = ctrl.Consequent(np.arange(-1,2,0.05),'out_velo')
    out_velo_delta.defuzzify_method = 'centroid'
    in_nw['low'] = fuzz.trimf(in_nw.universe,[1.5,1.5,1.7])
    in_nw['medium'] = fuzz.trimf(in_nw.universe,[1.5,1.7,2.5])
    in_nw['high'] = fuzz.trimf(in_nw.universe,[1.7,2.5,2.5])
    out_velo_delta['low'] = fuzz.trimf(out_velo_delta.universe,[-0.12,-0.12,0])

    out_velo_delta['medium'] = fuzz.trimf(out_velo_delta.universe,[-0.12,0,1.2])

    out_velo_delta['high'] = fuzz.trimf(out_velo_delta.universe,[0,1.2,2])
    
    acc_rule = ctrl.Rule(antecedent= in_nw['low'] ,consequent = out_velo_delta['low'])

    rul0 = ctrl.Rule(antecedent=(in_nw['medium'] ),consequent=out_velo_delta['medium'])

    dec_rule = ctrl.Rule(antecedent=in_nw['high'],consequent=out_velo_delta['high'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [75]:
step2 = rotate_phase_logic()

In [76]:
def rotate_tank(step = step2):
    tank.forward(0)
    tank.forward(3)
    tank.turn_left()
    t = time.time()

    NW = 0
    previous_NW = 10
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
            if(err_code == 0): 
                if sensor_name =='NW':
               
                    NW = np.linalg.norm(detectedPoint)
                   
            if (time.time() - t > 1)& (previous_NW > NW):
                t = time.time()
                print("Proximity_sensor WN",NW,"Velocity",tank.rightvelocity)
                step.input['in_nw'] = NW
                step.compute()
                avg_speed = (tank.rightvelocity+tank.leftvelocity)/2
                new_speeed = avg_speed +step.output['out_velo']
                if new_speeed <0.4:
                    tank.stop()
                    tank.forward(0)
                    return
                tank.forward(new_speeed)
                tank.turn_left(step.output['out_velo'])
                
              
    

In [77]:

def begin_parking_logic():
    # for now let's ignore spped 
    
   
    # rules
    in_ne = ctrl.Antecedent(np.arange(0.7,1.6,0.05),'in_ne')
    out_velo_delta = ctrl.Consequent(np.arange(-2,1,0.05),'out_velo')
    out_velo_delta.defuzzify_method = 'centroid'
    in_ne['low'] = fuzz.trimf(in_ne.universe,[0.7,0.7,1.1])
    in_ne['medium'] = fuzz.trimf(in_ne.universe,[1.1,1.2,1.6])
    in_ne['high'] = fuzz.trimf(in_ne.universe,[1.2,1.6,1.6])
    out_velo_delta['low'] = fuzz.trimf(out_velo_delta.universe,[-2,-2,-1])

    out_velo_delta['medium'] = fuzz.trimf(out_velo_delta.universe,[-2,-1,2])

    out_velo_delta['high'] = fuzz.trimf(out_velo_delta.universe,[-1,2,2])
    
    acc_rule = ctrl.Rule(antecedent= in_ne['low'] ,consequent = out_velo_delta['low'])

    rul0 = ctrl.Rule(antecedent=(in_ne['medium'] ),consequent=out_velo_delta['medium'])

    dec_rule = ctrl.Rule(antecedent=in_ne['high'],consequent=out_velo_delta['high'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [78]:
step3 = begin_parking_logic()

In [79]:
def begin_parking(step = step3):
    # tank.forward(0)
    tank.forward(2.05)
    # tank.turn_left()
    t = time.time()

    NE = 0
    previous_NE = 10
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
            if(err_code == 0): 
                if sensor_name =='NE':
               
                    NE = np.linalg.norm(detectedPoint)
                   
            if (time.time() - t > 0.5)& (previous_NE > NE):
                t = time.time()
                print("Proximity_sensor NE",NE,"Velocity",tank.rightvelocity)
                step.input['in_ne'] = NE
                step.compute()
                new_speeed = tank.rightvelocity +step.output['out_velo']
                if new_speeed <0:
                    tank.stop()
                    tank.forward(0)
                    return
                tank.forward(new_speeed)
                
                

In [80]:
def rotate_in_place():
    tank.forward(0)
    # tank.forward(2)
    tank.turn_left()
    counter = 0

    NW = 10
    previous_NW = 1.1
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
            if(err_code == 0): 
                counter +1
                previous_NW = NW
                if sensor_name =='NW':
                    
                    NW = np.linalg.norm(detectedPoint)
                    if counter % 100 == 0: 
                        print("Proxmity sensor NW",NW)
            if (NW <1.092) & (previous_NW > NW) & (previous_NW!=1.1):
                
                print("Rotating completed")
                tank.stop()
                tank.forward(0)
                return
                

In [81]:

def last_phase_logic():
    # for now let's ignore spped 
    
   
    # rules
    in_nw = ctrl.Antecedent(np.arange(0.5,1.2,0.05),'in_new')
    out_velo_delta = ctrl.Consequent(np.arange(0,4,0.05),'out_velo')
    out_velo_delta.defuzzify_method = 'centroid'
    in_nw['low'] = fuzz.trimf(in_nw.universe,[0.5,0.5,0.7])
    in_nw['medium'] = fuzz.trimf(in_nw.universe,[0.7,0.7,1])
    in_nw['high'] = fuzz.trimf(in_nw.universe,[1,1.5,1.5])
    out_velo_delta['low'] = fuzz.trimf(out_velo_delta.universe,[0,0,1])

    out_velo_delta['medium'] = fuzz.trimf(out_velo_delta.universe,[1,1.5,2])

    out_velo_delta['high'] = fuzz.trimf(out_velo_delta.universe,[1.5,2,4])
    
    acc_rule = ctrl.Rule(antecedent= in_nw['low'] ,consequent = out_velo_delta['low'])

    rul0 = ctrl.Rule(antecedent=(in_nw['medium'] ),consequent=out_velo_delta['medium'])

    dec_rule = ctrl.Rule(antecedent=in_nw['high'],consequent=out_velo_delta['high'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [82]:
step4 = last_phase_logic()

In [83]:
def last_phase(step = step4):
    counter = 0
    NW = 0
    Previous_NW = 10
    # previous_WN = 10
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
            if(err_code == 0): 
                if sensor_name =='NW':
               
                    NW = np.linalg.norm(detectedPoint)
                   
        
        counter +=1 
        if counter % 10000== 0:
            print("Proximity_sensor NW",NW,"Velocity",tank.rightvelocity)
            
                            
        try:
            if Previous_NW > NW:
                Previous_NW = NW
                step.input['in_new'] = NW
                step.compute()
                tank.forward(step.output['out_velo'])
                if (step.output['out_velo']<0.355) :
                    print("Stopping the tank")
                    tank.stop()
                    return
        except:
            continue

In [84]:
detect_spot()


Proximity_sensor WN 2.245449866189043 Velocity 3.507551252864972
Proximity_sensor WS 2.245449866189043 Velocity 3.507551252864972
Proximity_sensor WN 2.1021501814066395 Velocity 3.1825949886579075
Proximity_sensor WS 2.1021501814066395 Velocity 3.1825949886579075
Proximity_sensor WN 2.0018648727420443 Velocity 2.943253152824983
Proximity_sensor WS 2.0018648727420443 Velocity 2.943253152824983
Proximity_sensor WN 1.9807595963410378 Velocity 2.889578231831319
Proximity_sensor WS 1.9798440778333217 Velocity 2.889578231831319
Proximity_sensor WN 1.9299761323308802 Velocity 2.7533407565417516
Proximity_sensor WS 1.9124562375682994 Velocity 2.7533407565417516
Proximity_sensor WN 1.9969177120772075 Velocity 2.9308043642610606
Proximity_sensor WS 1.9541349299043647 Velocity 2.9308043642610606
Proximity_sensor WN 1.934092951428764 Velocity 2.764816137673802
Proximity_sensor WS 1.933247545880616 Velocity 2.764816137673802


In [85]:
rotate_tank()


Proximity_sensor WN 2.4752699959642324 Velocity 4
Proximity_sensor WN 2.122426996641429 Velocity 1.0521645039156051
Proximity_sensor WN 2.03373411580981 Velocity 0.8722750897872955
Proximity_sensor WN 2.0007343339659305 Velocity 0.7952389957470948
Proximity_sensor WN 1.8418207882943072 Velocity 0.763251829334958
Proximity_sensor WN 1.730085783980095 Velocity 0.5792581817730551
Proximity_sensor WN 1.708295591455422 Velocity 0.40945437486690894


In [86]:
begin_parking()


Proximity_sensor NE 1.8682324078366812 Velocity 2.05
Proximity_sensor NE 1.7251577451281923 Velocity 2.0465803631472816
Proximity_sensor NE 1.6381269619583343 Velocity 2.0431607262945635
Proximity_sensor NE 1.4980762565184043 Velocity 2.0397410894418453
Proximity_sensor NE 1.386604574907212 Velocity 1.8273676252409239
Proximity_sensor NE 1.3061696718876337 Velocity 1.407292350185531
Proximity_sensor NE 1.2461844618859357 Velocity 0.9360034565177933
Proximity_sensor NE 1.226290492901734 Velocity 0.4437495352343938


In [87]:
rotate_in_place()

Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sensor NW 1.2624523408171777
Proxmity sen

In [88]:
last_phase()

Proximity_sensor NW 1.0428210914879863 Velocity 2.7145444857499963
Proximity_sensor NW 0.9867430841974265 Velocity 1.5000000000000415
Proximity_sensor NW 0.9269632198615679 Velocity 1.4999999999999998
Proximity_sensor NW 0.8917445508011561 Velocity 1.4999999999999998
Proximity_sensor NW 0.8533749835773308 Velocity 1.4999999999999998
Proximity_sensor NW 0.8054019077867398 Velocity 1.4999999999999998
Proximity_sensor NW 0.7841867702295503 Velocity 1.4999999999999998
Proximity_sensor NW 0.7482150133250433 Velocity 1.4999999999999996
Proximity_sensor NW 0.7628340638592044 Velocity 1.4999999999999998
Proximity_sensor NW 0.7530223547428052 Velocity 1.4999999999999998
Proximity_sensor NW 0.7303234673251429 Velocity 1.4999999999999998
Proximity_sensor NW 0.6785230834096145 Velocity 1.2952453097063683
Proximity_sensor NW 0.6935196325507911 Velocity 1.2952453097063683
Proximity_sensor NW 0.7170971936869329 Velocity 1.2952453097063683
Proximity_sensor NW 0.7383872578944906 Velocity 1.295245309706